## Task 2 Artificial Neuron Network (ANN) in Keras

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

### Discover and visualize data file `train.csv`
1. Read in the data file using pandas

In [2]:
dataframe = pd.read_csv("../magic04.data")
dataframe

,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092,81.8828,g
0,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
1,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
2,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
3,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g
4,51.6240,21.1502,2.9085,0.2420,0.1340,50.8761,43.1887,9.8145,3.6130,238.0980,g
...,...,...,...,...,...,...,...,...,...,...,...
19014,21.3846,10.9170,2.6161,0.5857,0.3934,15.2618,11.5245,2.8766,2.4229,106.8258,h
19015,28.9452,6.7020,2.2672,0.5351,0.2784,37.0816,13.1853,-2.9632,86.7975,247.4560,h
19016,75.4455,47.5305,3.4483,0.1417,0.0549,-9.3561,41.0562,-9.4662,30.2987,256.5166,h
19017,120.5135,76.9018,3.9939,0.0944,0.0683,5.8043,-93.5224,-63.8389,84.6874,408.3166,h


2. Display information of the dataset that is just read in

In [3]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19019 entries, 0 to 19018
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   28.7967  19019 non-null  float64
 1   16.0021  19019 non-null  float64
 2   2.6449   19019 non-null  float64
 3   0.3918   19019 non-null  float64
 4   0.1982   19019 non-null  float64
 5   27.7004  19019 non-null  float64
 6   22.011   19019 non-null  float64
 7   -8.2027  19019 non-null  float64
 8   40.092   19019 non-null  float64
 9   81.8828  19019 non-null  float64
 10  g        19019 non-null  object 
dtypes: float64(10), object(1)
memory usage: 1.6+ MB


3. Display important statistic attribute of all features to see whether to drop any not column that has large differences in the values

In [4]:
dataframe.describe()

,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.011,-8.2027,40.092,81.8828
count,19019.000000,19019.000000,19019.000000,19019.000000,19019.000000,19019.000000,19019.000000,19019.000000,19019.000000,19019.000000
mean,53.251440,22.181291,2.825026,0.380326,0.214658,-4.333429,10.544942,0.250170,27.645052,193.823912
std,42.365598,18.346484,0.472609,0.182818,0.110514,59.207163,51.001391,20.827896,26.104151,74.729344
min,4.283500,0.000000,1.941300,0.013100,0.000300,-457.916100,-331.780000,-205.894700,0.000000,1.282600
25%,24.336000,11.863700,2.477100,0.235800,0.128450,-20.588300,-12.845050,-10.849750,5.546950,142.499000
50%,37.149000,17.140600,2.739600,0.354100,0.196500,4.011900,15.309400,0.689800,17.677000,191.856900
75%,70.126850,24.739950,3.101600,0.503700,0.285250,24.060350,35.844100,10.947050,45.884100,240.564550
max,334.177000,256.382000,5.323300,0.893000,0.675200,575.240700,238.321000,179.851000,90.000000,495.561000


4. Print out the correlation of the features and the target to find out their relevance

In [5]:
dataframe.corr()

ValueError: could not convert string to float: 'g'

### Visualize data
#### Visualize the target class values to see if there is any abnormal distribution

In [6]:
plt.hist(dataframe['critical_temp'], edgecolor='maroon', facecolor='orangered')
plt.xlabel("Temperature (Celsius)")
plt.ylabel("Number of temperature label")
plt.title("Temperature of for the target class")
plt.show()

KeyError: 'critical_temp'

#### Visualize the number of elements feature to see if there is any abnormal distribution

In [ ]:
plt.hist(dataframe['number_of_elements'], edgecolor='maroon', facecolor='orangered')
plt.xlabel("Numbers of elements")
plt.ylabel("Frequency")
plt.title("Features numbers of elements data distribution")
plt.show()

#### Checking null value in dataset

In [ ]:
dataframe.isnull().sum()

#### Create a copy of dataset, then split into 2/3 for training and the rest for testing

In [ ]:
df = dataframe.copy()

X = df.drop(columns=['critical_temp'])
y = df['critical_temp']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

#### Feature Scaling
Since it is a linear regression problem ( which aims to find the best-fitting line that describes the relationship between the independent variables (features) and the dependent variable (target)), scaling all feature and the target class to optimize the algorithm of finding the best fitting line and also help the learning model to be trained faster.
1. Using MinMaxScaler()
2. fit into the object of this scaler training features dataset
3. Using that standard to scale the testing features

In [7]:
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train

NameError: name 'X_train' is not defined

**Transform the target with same method of fitting the training label to the scaler
and scale the testing target**

In [ ]:
scaler.fit(y_train.values.reshape(-1,1))
y_train = scaler.transform(y_train.values.reshape(-1,1))
y_train = y_train.reshape(-1)
y_train = pd.Series(y_train)
y_test = scaler.transform(y_test.values.reshape(-1,1))
y_test = y_test.reshape(-1)
y_test = pd.Series(y_test)
y_test

Splitting the training data set into two subsets, one for training and one for validation

In [ ]:
x_val = X_train[:3000]
partial_X_train = X_train[3000:]
y_val = y_train[:3000]
partial_y_train = y_train[3000:]
partial_X_train

#### Create a function to build a artificial neuron network using Keras API
1. `build_model()` function takes in for parameters
    * Number of hidden layers : `n_hidden`
    * Number of neurons of each layer : `n_neurons`
    * Regularization number (L1, L2): `a`, `b`
2. Model using the optimizers of `RMSprop` with a `learning_rate` equals 0.0002
3. Model using Mean Squared Error as a loss function

In [ ]:
from keras import regularizers
from keras.applications.densenet import layers
from keras import metrics

optimizer=keras.optimizers.Adam()
def build_model(n_hidden= 3, n_neurons = 32, a = 0.001, b=0.002):
    model = keras.Sequential()
    for layer in range(n_hidden):
        model.add(layers.Dense(n_neurons, activation = 'relu', kernel_regularizer=regularizers.L1L2(l1=a, l2=b)))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizer, loss=keras.losses.MeanSquaredError(), metrics=[metrics.MeanSquaredError()])
    return model

**Fitting a model training data subset**
* With 60 iterations and a `batch_size` for dataset of 512
* Print out two loss with a validation dataset when fit into model

In [ ]:
model = build_model()
history = model.fit(partial_X_train,
                    partial_y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(x_val, y_val))

**Predict a temperature for a testing data using the trained model above and print out the results**
* Inserse the prediction into original scale to be friendlier in reading the data temperature

#### Visualize the two loss during fitting into the model to check the efficiency of the model

In [ ]:
mse = history.history['mean_squared_error']
var = np.std(y)**2
def r_squared(mse, var):
    return (1-mse/var)

for i in range(len(mse)):
    print(r_squared(mse[i], var))

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1, len(loss_values) + 1)
plt.plot(epochs, loss_values, "bo", label="Training loss")
plt.plot(epochs, val_loss_values, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['mean_squared_error'])

In [ ]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
predictions

In [ ]:
score = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {score[1]:.4f}')

### Fine tune model
To make the model become more accurate by implementing:
* Fine tune some hyperparameter of number of hidden layers `n_hidden`, numeber of neurons `n_neurons`
* Changing the regularization L1 and L2 to reduce the loss and not become overfitting

In [ ]:
param_distribs = {
    'n_hidden': [2,4,6],
    'n_neurons': [32,64],
    'a': [0.0002, 0.001],
    'b': [0.0002, 0.001]
}

#### Build a model and choose the best hyperparameter

In [ ]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [ ]:
search_cv = GridSearchCV(keras_reg, param_distribs, cv = 3)
search_cv.fit(partial_X_train, partial_y_train, epochs=60,validation_data=(x_val,y_val), \
              callbacks=[keras.callbacks.EarlyStopping(patience=30)])
search_cv.best_params_
search_cv.best_score_

**Print out the best parameters after hypertuning**

In [ ]:
search_cv.best_params_

In [ ]:
best_model = search_cv.best_estimator_.model
best_model.summary()

#### evaluating the model and making predictions after hypertuning

In [ ]:
best_model.evaluate(X_train, y_train)
best_model.predict(X_train)

In [ ]:
best_model.evaluate(np.array(X_test),np.array(y_test))
best_model.predict(X_test)